In [ ]:
#データベース作成、ベクトル埋め込み
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import os

# パラメータ
JSONL_FILE = "paragraphs.jsonl"
INDEX_FILE = "faiss_index.bin"
NPZ_FILE = "faiss_metadata.npz"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
EMB_DIM = 384

# モデルロード
model = SentenceTransformer(MODEL_NAME)

# 既存のFAISS & npz 読み込み（なければ新規作成）
if os.path.exists(INDEX_FILE) and os.path.exists(NPZ_FILE):
    print("既存のFAISSインデックスとメタデータを読み込みます...")
    index = faiss.read_index(INDEX_FILE)
    npz_data = np.load(NPZ_FILE, allow_pickle=True)
    metadata_list = npz_data["metadata_list"].tolist()
    paragraphs = npz_data["paragraphs"].tolist()
    existing_ids = {m["id"] for m in metadata_list}
else:
    print("新規にインデックスとメタデータを作成します...")
    index = faiss.IndexFlatIP(EMB_DIM)
    metadata_list = []
    paragraphs = []
    existing_ids = set()

# JSONL から新規データだけ取得
new_paragraphs = []
new_metadata = []
new_ids = []

with open(JSONL_FILE, "r", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line)
        _id = entry["id"]
        if _id in existing_ids:
            continue  # すでに登録済み
        new_ids.append(_id)
        new_paragraphs.append(entry["paragraph"])
        new_metadata.append({
            "id": _id,
            "title": entry["title"],
            "page": entry["metadata"].get("page"),
            "page": entry["metadata"].get("author"),
            "source": entry["metadata"].get("source")
        })

print(f"新規に追加する段落数: {len(new_paragraphs)}")

if new_paragraphs:
    # ベクトル化
    embeddings = model.encode(new_paragraphs, batch_size=32, show_progress_bar=True)
    embeddings = np.array(embeddings).astype("float32")
    faiss.normalize_L2(embeddings)

    # FAISS に追加
    index.add(embeddings)

    # リスト更新
    paragraphs.extend(new_paragraphs)
    metadata_list.extend(new_metadata)

    # 保存
    faiss.write_index(index, INDEX_FILE)
    np.savez(NPZ_FILE, paragraphs=paragraphs, metadata_list=metadata_list)
    print("FAISS & .npz を差分更新しました。")
else:
    print("差分はありません。更新はスキップされました。")



In [ ]:
#平均ベクトル作成用update_npz.py
import numpy as np
import faiss
import os

# === パラメータ ===
INDEX_FILE = "faiss_index.bin"
NPZ_FILE = "faiss_metadata.npz"
EMB_DIM = 384

# === FAISSインデックス読み込み ===
print("FAISSインデックスを読み込み中...")
index = faiss.read_index(INDEX_FILE)
total_vecs = index.ntotal

# === ベクトルを再構成（FAISS内部から取り出す） ===
print(f"{total_vecs} 個のベクトルを再構成中...")
reconstructed = np.empty((total_vecs, EMB_DIM), dtype='float32')
index.reconstruct_n(0, total_vecs, reconstructed)

# === 既存のnpzデータ読み込み ===
print("既存の npz を読み込み中...")
data = np.load(NPZ_FILE, allow_pickle=True)
paragraphs = data["paragraphs"].tolist()
metadata_list = data["metadata_list"].tolist()

# === 整合性チェック ===
if len(paragraphs) != total_vecs or len(metadata_list) != total_vecs:
    raise ValueError(f"段落数({len(paragraphs)})とベクトル数({total_vecs})が一致しません。")

# === 新しい npz に保存（上書き）===
print("embeddings を追加して npz を保存します...")
np.savez(
    NPZ_FILE,
    paragraphs=paragraphs,
    metadata_list=metadata_list,
    embeddings=reconstructed
)

print("更新完了：embeddings が npz に保存されました。")


FAISSインデックスを読み込み中...
35299 個のベクトルを再構成中...
既存の npz を読み込み中...
embeddings を追加して npz を保存します...
更新完了：embeddings が npz に保存されました。
